# MSA 2024 Phase 2 - Part 3

Welcome to the competition - in Part 3, you are encouraged to utilize neural network based models for classification.

This notebook builds a simple Multi-Layer Perceptron (MLP) model for the CIFAR-10 dataset, with the use of `keras` to define the model structure.

**Before start working on the competition, please ensure all required libraries are installed and properly set up on your system**:

- `python >= 3.6`,
- `tensorFlow >= 2.0`,
- `keras >= 2.3`,

and any neccassary liburaries for data manipulation and processing, e.g., `numpy`, `pandas`, etc.

In [60]:
import tensorflow as tf
import numpy as np

### 1. Data loading & preprocessing

The CIFAR-10 dataset contains 60,000 images(32x32x3) in 10 different classes, with 6,000 images in each class. You can download the dataset directly from the competition webpage.

**To train the model, you are expected to use the training label provided in train.csv**.

In [58]:
import zipfile
import os

# Prepare the training and testing image sets.
def unzipDataset(data_dir):
    zip_path = data_dir + '.zip'
    extract_path = os.getcwd()

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

train_dir = 'train'
test_dir = 'test'

unzipDataset(train_dir)
unzipDataset(test_dir)

In [114]:
from PIL import Image

def loadTrain(root_dir, csv_file):
    ids = []
    images = []
    labels = []
    annotations = np.genfromtxt('train.csv', delimiter=',', names=True)
    for idx in range(len(annotations)):
        img_id = int(annotations['id'][idx])
        img_name = os.path.join(root_dir, f"image_{img_id}.png")
        image = np.array(Image.open(img_name).convert("RGB"))
        label = int(annotations['label'][idx])

        ids.append(img_id)
        images.append(image)
        labels.append(label)
    return np.array(ids), np.array(images), np.array(labels)

def loadTest(root_dir):
    ids = []
    images = []
    for idx in range(len(os.listdir(root_dir))):
        img_name = os.path.join(root_dir, f"image_{idx}.png")
        image = np.array(Image.open(img_name).convert("RGB"))

        ids.append(idx)
        images.append(image)
    return np.array(ids), np.array(images)


# Load training, testing data and the training label provided in train.csv.
train_csv = 'train.csv'
id_train, X_train, y_train = loadTrain(train_dir, train_csv)
id_test, X_test = loadTest(test_dir)

# Normalize the data. Reshape the data to fit in to an MLP model.
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
# X_train = X_train.reshape(-1, 3072)
# X_test = X_test.reshape(-1, 3072)

def to_categorical(y, num_classes=None):
    """
    Converts a class vector (integers) to binary class matrix (one-hot encoding).

    Arguments:
    y -- Class vector to be converted into a matrix (integers).
    num_classes -- Total number of classes (optional, inferred from y if not provided).

    Returns:
    One-hot encoded matrix.
    """
    y = np.array(y, dtype='int')
    if num_classes is None:
        num_classes = np.max(y) + 1
    categorical = np.zeros((len(y), num_classes))
    categorical[np.arange(len(y)), y] = 1
    return categorical
# Convert training labels to one-hot encoded vectors.
y_train = to_categorical(y_train, 10)

array([[0.6156863 , 0.61960787, 0.62352943, ..., 0.7411765 , 0.6784314 ,
        0.6392157 ],
       [0.8       , 0.84705883, 0.8666667 , ..., 0.78039217, 0.7607843 ,
        0.78039217],
       [1.        , 1.        , 1.        , ..., 0.654902  , 0.6666667 ,
        0.6627451 ],
       ...,
       [0.5686275 , 0.84705883, 0.9607843 , ..., 0.52156866, 0.56078434,
        0.59607846],
       [0.87058824, 0.9137255 , 0.96862745, ..., 0.88235295, 0.87058824,
        0.8352941 ],
       [0.9490196 , 0.9490196 , 0.96862745, ..., 0.81960785, 0.81960785,
        0.8156863 ]], dtype=float32)

In [62]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
X_test shape: (5000, 32, 32, 3)


### 2. Build & train the model

This code demostrates a simple Multi-Layer Perceptron (MLP) model. However, you are encouraged to experiment with more complex deep learning models and techniques to boost your performance.

In [63]:
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Model initialization.
model = tf.keras.Sequential()
 
# Build the MLP model.
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(128, activation='relu'))
model.add(Dense(10,  activation="softmax"))

# Complile the model.
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model.
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Make predictions.
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)

# Prepare your submission file.
submission = np.column_stack((id_test, predicted_labels))
np.savetxt('submission.csv', submission, delimiter=',', header='id, Label', comments='', fmt='%d')

/Users/vitaliigalkin/Desktop/MSA-Phase2/.venv/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(32, 32, 32, 3), dtype=float32). Expected shape (None, 32), but input has incompatible shape (32, 32, 32, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(32, 32, 32, 3), dtype=float32)
  • training=True
  • mask=None

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import classification_report, confusion_matrix

INPUT_SHAPE = (32, 32, 3)
KERNEL_SIZE = (3, 3)
model = Sequential()

# Convolutional Layer
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
# Pooling layer
model.add(MaxPool2D(pool_size=(2, 2)))
# Dropout layers
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=KERNEL_SIZE, input_shape=INPUT_SHAPE, activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
# model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(10, activation='softmax'))

METRICS = [
    'accuracy',
    tf.keras.metrics.Precision(name='precision'),
    tf.keras.metrics.Recall(name='recall')
]
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=METRICS)

/Users/vitaliigalkin/Desktop/MSA-Phase2/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [65]:
model.summary()

Model: "sequential_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_15 (Conv2D)              │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_12          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 32, 32, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_13          │ (None, 32, 32, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 16, 16, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_14          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_18 (Conv2D)              │ (None, 16, 16, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_15          │ (None, 16, 16, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 8, 8, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_19 (Conv2D)              │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_16          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 8, 8, 128)      │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_17          │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_19 (Dropout)            │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_30 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_20 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_31 (Dense)                │ (None, 10)             │         1,29

 Total params: 552,362 (2.11 MB)

 Trainable params: 551,466 (2.10 MB)

 Non-trainable params: 896 (3.50 KB)

In [66]:
early_stop = EarlyStopping(monitor='val_loss', patience=5)

In [72]:
from sklearn.model_selection import train_test_split


batch_size = 32
X_new_train, X_val, y_new_train, y_val = train_test_split(X_train,y_train,test_size=0.2, random_state=1214)

data_generator = ImageDataGenerator(width_shift_range=0.1, height_shift_range=0.1, horizontal_flip=True)
train_generator = data_generator.flow(X_new_train, y_new_train, batch_size)
steps_per_epoch = X_train.shape[0] // batch_size



r = model.fit(train_generator, 
              epochs=50,
              steps_per_epoch=steps_per_epoch,
              validation_data=(X_val, y_val), 
              callbacks=[early_stop],
#               batch_size=batch_size,
             )

Epoch 1/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.8951 - loss: 0.3059 - precision: 0.9202 - recall: 0.8731 - val_accuracy: 0.8699 - val_loss: 0.4323 - val_precision: 0.8957 - val_recall: 0.8518
Epoch 2/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.8897 - loss: 0.3223 - precision: 0.9164 - recall: 0.8664 - val_accuracy: 0.8529 - val_loss: 0.4814 - val_precision: 0.8794 - val_recall: 0.8337
Epoch 3/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 80s 51ms/step - accuracy: 0.8945 - loss: 0.3066 - precision: 0.9209 - recall: 0.8724 - val_accuracy: 0.8247 - val_loss: 0.5705 - val_precision: 0.8599 - val_recall: 0.8012
Epoch 4/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 81s 52ms/step - accuracy: 0.8947 - loss: 0.3092 - precision: 0.9212 - recall: 0.8698 - val_accuracy: 0.8782 - val_loss: 0.3957 - val_precision: 0.8997 - val_recall: 0.8609
Epoch 5/50
1562/1562 ━━━━━━━━━━━━━━━━━━━━ 82s 52ms/step - accuracy: 0.8920 - loss: 0.3080 - precision: 0.9180 - recall: 0.8705 - val_accuracy: 0

In [84]:
from tensorflow.keras.applications.resnet50 import ResNet50
local_weights_path = '/Users/vitaliigalkin/Downloads/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
convolutional_base = ResNet50(weights='imagenet',include_top=False, input_shape=INPUT_SHAPE)

In [124]:
from tensorflow.keras import models, layers,optimizers
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
local_weights_path = '/Users/vitaliigalkin/Downloads/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'
convolutional_base = ResNet50(weights='imagenet',include_top=False, input_shape=(256, 256, 3))
# Freeze the convolutional base
convolutional_base.trainable = False

# Define the model
model = models.Sequential()
model.add(layers.InputLayer(input_shape=INPUT_SHAPE))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.UpSampling2D((2, 2)))
model.add(layers.UpSampling2D((2, 2)))
model.add(convolutional_base)
model.add(layers.Flatten())
model.add(layers.BatchNormalization())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())
model.add(layers.Dense(num_of_classes, activation='softmax'))

# Compile the model
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=0.001),
#     loss='categorical_crossentropy',  # use 'sparse_categorical_crossentropy' if labels are integers
#     metrics=['accuracy']
# )

# Compile the model
optimizer = optimizers.RMSprop(learning_rate=2e-5)
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 17s 0us/step


In [125]:
model.summary()

Model: "sequential_23"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ up_sampling2d_30 (UpSampling2D) │ (None, 64, 64, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_31 (UpSampling2D) │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ up_sampling2d_32 (UpSampling2D) │ (None, 256, 256, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 8, 8, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_45          │ (None, 131072)         │       524,288 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 128)            │    16,777,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_46          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_47          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 10)             │           650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 40,899,018 (156.02 MB)

 Trainable params: 17,048,778 (65.04 MB)

 Non-trainable params: 23,850,240 (90.98 MB)

In [130]:
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train dtype:", X_train.dtype)
print("y_train dtype:", y_train.dtype)
y_train = y_train.astype(np.float32)

history = model.fit(X_train, y_train, batch_size=100,validation_split=0.2, epochs=10)

X_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 10)
X_train dtype: float32
y_train dtype: float32
Epoch 1/10
 14/400 ━━━━━━━━━━━━━━━━━━━━ 20:08 3s/step - accuracy: 0.2262 - loss: 2.3193

KeyboardInterrupt: 

In [68]:
predic = model.predict(X_test)
predict_labels = np.argmax(predic, axis=1)

157/157 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step


In [73]:
print(predict_labels)

[0 0 2 ... 2 9 9]


In [70]:
submission = np.column_stack((id_test, predict_labels))
np.savetxt('submission.csv', submission, delimiter=',', header='id, Label', comments='', fmt='%d')